# Intro

Deze notebook toont de Python-implementatie van het protocol energiedelen versie 3.1

Het protocol beschrijft 3 verdeelsleutels: Vast, Relatief en Optimaal.

In [1]:
import pandas as pd

In [2]:
from openenergyid import TimeDataFrame
from openenergyid.energysharing import EnergySharingInput, EnergySharingOutput, calculate, CalculationMethod

## Input Data

Om de werking aan te tonen, maken we een sample dataset met 6 deelnemers, gedurende 4 kwartieren.

Elke deelnemer heeft een Bruto Afname en een Bruto Injectie. Voor elk kwartier is er een verdeelsleutel, die bepaalt hoeveel energie er naar elke deelnemer gaat. We wijzen een verdeelsleutel per kwartier to, omdat deze verdeelsleutels in de tijd kunnen veranderen.

Het resultaat zal zijn dat elke deelnemer een Netto Afname, een Netto Injectie en een hoeveelheid gedeelde energie heeft.

In [3]:
index = pd.date_range(
    start=pd.Timestamp('2024-01-01 12:00:00', tz='Europe/Brussels'),
    end=pd.Timestamp('2024-01-01 12:45:00', tz='Europe/Brussels'),
    freq='15T',
)

In [4]:
gross_injection = pd.DataFrame(
    data={
        "P1": [70, 20, 100, 0],
        "P2": [10, 10, 10, 10],
        "P3": [20, 30, 40, 50],
        "P4": [0, 0, 0, 0],
        "P5": [0, 0, 0, 0],
        "P6": [0, 0, 0, 0],
    },
    index=index,
)

gross_offtake = pd.DataFrame(
    data={
        "P1": [0, 0, 0, 0],
        "P2": [30, 20, 10, 0],
        "P3": [22, 20, 30, 40],
        "P4": [30, 25, 35, 10],
        "P5": [18, 17, 19, 25],
        "P6": [20, 22, 24, 26],
    },
    index=index,
)

key = pd.DataFrame(
    data={
        "P1": [0, 0, 0, 0],
        "P2": [10, 10, 10, 10],
        "P3": [22.5, 22.5, 22.5, 22.5],
        "P4": [22.5, 22.5, 22.5, 22.5],
        "P5": [22.5, 22.5, 22.5, 22.5],
        "P6": [22.5, 22.5, 22.5, 22.5],
    },
    index=index,
)
key = key.div(100)

In [5]:
gross_injection

,P1,P2,P3,P4,P5,P6
2024-01-01 12:00:00+01:00,70,10,20,0,0,0
2024-01-01 12:15:00+01:00,20,10,30,0,0,0
2024-01-01 12:30:00+01:00,100,10,40,0,0,0
2024-01-01 12:45:00+01:00,0,10,50,0,0,0


In [6]:
gross_offtake

,P1,P2,P3,P4,P5,P6
2024-01-01 12:00:00+01:00,0,30,22,30,18,20
2024-01-01 12:15:00+01:00,0,20,20,25,17,22
2024-01-01 12:30:00+01:00,0,10,30,35,19,24
2024-01-01 12:45:00+01:00,0,0,40,10,25,26


In [7]:
key

,P1,P2,P3,P4,P5,P6
2024-01-01 12:00:00+01:00,0.0,0.1,0.225,0.225,0.225,0.225
2024-01-01 12:15:00+01:00,0.0,0.1,0.225,0.225,0.225,0.225
2024-01-01 12:30:00+01:00,0.0,0.1,0.225,0.225,0.225,0.225
2024-01-01 12:45:00+01:00,0.0,0.1,0.225,0.225,0.225,0.225


In [8]:
# Construct the input model
# This helps with validation and serialization

input = EnergySharingInput.model_construct(
    gross_injection=TimeDataFrame.from_pandas(gross_injection),
    gross_offtake=TimeDataFrame.from_pandas(gross_offtake),
    key=TimeDataFrame.from_pandas(key),
)

df = input.data_frame()

In [9]:
df

Gross Injection                  Gross Offtake      \
                                       P1  P2  P3 P4 P5 P6            P1  P2   
2024-01-01 12:00:00+01:00              70  10  20  0  0  0             0  30   
2024-01-01 12:15:00+01:00              20  10  30  0  0  0             0  20   
2024-01-01 12:30:00+01:00             100  10  40  0  0  0             0  10   
2024-01-01 12:45:00+01:00               0  10  50  0  0  0             0   0   

                                           Key                            \
                           P3  P4  P5  P6   P1   P2     P3     P4     P5   
2024-01-01 12:00:00+01:00  22  30  18  20  0.0  0.1  0.225  0.225  0.225   
2024-01-01 12:15:00+01:00  20  25  17  22  0.0  0.1  0.225  0.225  0.225   
2024-01-01 12:30:00+01:00  30  35  19  24  0.0  0.1  0.225  0.225  0.225   
2024-01-01 12:45:00+01:00  40  10  25  26  0.0  0.1  0.225  0.225  0.225   

                                  
                              P6  
2024-01-01 12:00:00+01:00  0.225  
2024-01-01 12:15:00+01:00  0.225  
2024-01-01 12:30:00+01:00  0.225  
2024-01-01 12:45:00+01:00  0.225

# Vaste methode

![Methodiek vaste methode](data/energiedelen/Vast.png)

In [10]:
calculate(df, method=CalculationMethod.FIXED)

Net Injection                              \
                                     P1    P2     P3   P4   P5   P6   
2024-01-01 12:00:00+01:00           4.9  1.70   5.90  0.0  0.0  0.0   
2024-01-01 12:15:00+01:00           0.0  1.00   6.75  0.0  0.0  0.0   
2024-01-01 12:30:00+01:00          19.0  2.90  16.60  0.0  0.0  0.0   
2024-01-01 12:45:00+01:00           0.0  2.42  18.33  0.0  0.0  0.0   

                          Net Offtake                                  \
                                   P1    P2     P3     P4    P5    P6   
2024-01-01 12:00:00+01:00         0.0  21.0   4.00   7.50   0.0   0.0   
2024-01-01 12:15:00+01:00         0.0  15.0  13.25  11.50   3.5   8.5   
2024-01-01 12:30:00+01:00         0.0   0.0   5.25   1.25   0.0   0.0   
2024-01-01 12:45:00+01:00         0.0   0.0  37.75   0.00  11.5  12.5   

                          Shared Energy                                  
                                     P1    P2     P3     P4    P5    P6  
2024-01-01 12:00:00+01:00           0.0   9.0  18.00  22.50  18.0  20.0  
2024-01-01 12:15:00+01:00           0.0   5.0   6.75  13.50  13.5  13.5  
2024-01-01 12:30:00+01:00           0.0  10.0  24.75  33.75  19.0  24.0  
2024-01-01 12:45:00+01:00           0.0   0.0   2.25  10.00  13.5  13.5

# Relatieve methode

![Methodiek relatieve methode](data/energiedelen/Relatief.png)

In [11]:
calculate(df, method=CalculationMethod.RELATIVE)

Net Injection                              \
                                     P1    P2     P3   P4   P5   P6   
2024-01-01 12:00:00+01:00          7.08  1.01   2.02  0.0  0.0  0.0   
2024-01-01 12:15:00+01:00          0.00  0.00  -0.00  0.0  0.0  0.0   
2024-01-01 12:30:00+01:00         24.67  2.47   9.87  0.0  0.0  0.0   
2024-01-01 12:45:00+01:00          0.00  2.24  11.22  0.0  0.0  0.0   

                          Net Offtake                                  \
                                   P1     P2     P3    P4    P5    P6   
2024-01-01 12:00:00+01:00         0.0  20.42   3.75  5.94  0.00  0.00   
2024-01-01 12:15:00+01:00         0.0  14.13  13.00  9.29  1.29  6.29   
2024-01-01 12:30:00+01:00         0.0   0.00   5.00  0.00  0.00  0.00   
2024-01-01 12:45:00+01:00         0.0   0.00  37.50  0.00  7.98  8.98   

                          Shared Energy                                     
                                     P1     P2     P3     P4     P5     P6  
2024-01-01 12:00:00+01:00           0.0   9.58  18.25  24.06  18.00  20.00  
2024-01-01 12:15:00+01:00           0.0   5.87   7.00  15.71  15.71  15.71  
2024-01-01 12:30:00+01:00           0.0  10.00  25.00  35.00  19.00  24.00  
2024-01-01 12:45:00+01:00           0.0   0.00   2.50  10.00  17.02  17.02

# Optimale methode

## Iteratie 1

![Methodiek optimale methode iteratie 1](data/energiedelen/Optimaal_1.png)

## Iteratie 2

![Methodiek optimale methode iteratie 2](data/energiedelen/Optimaal_2.png)

In [12]:
calculate(df, method=CalculationMethod.OPTIMAL)

Net Injection                              \
                                     P1    P2     P3   P4   P5   P6   
2024-01-01 12:00:00+01:00          0.00  0.00   0.00  0.0  0.0  0.0   
2024-01-01 12:15:00+01:00          0.00  0.00   0.00  0.0  0.0  0.0   
2024-01-01 12:30:00+01:00         14.76  1.48  15.77  0.0  0.0  0.0   
2024-01-01 12:45:00+01:00          0.00  0.00   0.00  0.0  0.0  0.0   

                          Net Offtake                                  \
                                   P1     P2     P3    P4    P5    P6   
2024-01-01 12:00:00+01:00         0.0  18.51   0.35  1.14  0.00  0.00   
2024-01-01 12:15:00+01:00         0.0  14.13  13.00  9.29  1.29  6.29   
2024-01-01 12:30:00+01:00         0.0   0.00   0.00  0.00  0.00  0.00   
2024-01-01 12:45:00+01:00         0.0   0.00  36.75  0.00  1.62  2.62   

                          Shared Energy                                     
                                     P1     P2     P3     P4     P5     P6  
2024-01-01 12:00:00+01:00           0.0  11.49  21.65  28.86  18.00  20.00  
2024-01-01 12:15:00+01:00           0.0   5.87   7.00  15.71  15.71  15.71  
2024-01-01 12:30:00+01:00           0.0  10.00  30.00  35.00  19.00  24.00  
2024-01-01 12:45:00+01:00           0.0   0.00   3.25  10.00  23.38  23.38

# Output Data

In [13]:
result = calculate(df, method=CalculationMethod.OPTIMAL)

In [14]:
output = EnergySharingOutput.from_calculation_result(result)

In [15]:
print(output.model_dump_json(by_alias=True, indent=2))

{
  "netInjection": {
    "index": [
      "2024-01-01T12:00:00+01:00",
      "2024-01-01T12:15:00+01:00",
      "2024-01-01T12:30:00+01:00",
      "2024-01-01T12:45:00+01:00"
    ],
    "columns": [
      "P1",
      "P2",
      "P3",
      "P4",
      "P5",
      "P6"
    ],
    "data": [
      [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
      ],
      [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
      ],
      [
        14.76,
        1.48,
        15.77,
        0.0,
        0.0,
        0.0
      ],
      [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
      ]
    ]
  },
  "netOfftake": {
    "index": [
      "2024-01-01T12:00:00+01:00",
      "2024-01-01T12:15:00+01:00",
      "2024-01-01T12:30:00+01:00",
      "2024-01-01T12:45:00+01:00"
    ],
    "columns": [
      "P1",
      "P2",
      "P3",
      "P4",
      "P5",
      "P6"
    ],
    "data": [
      [
        0.0,
        